# Sunburst diagram

In [169]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import streamlit as st
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    

    def find_pattern(self, query_string):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_pattern, query_string)
            for row in result:
                return row["cnt"]


    @staticmethod
    def _find_and_return_pattern(tx, query_string):
        result = tx.run(query_string+" as cnt")
        return [row for row in result]

In [173]:
def all_teams():
    return {'Austria',
     'Belgium',
     'Croatia',
     'Czech_Republic',
     'Denmark',
     'England',
     'Finland',
     'France',
     'Germany',
     'Hungary',
     'Italy',
     'Netherlands',
     'MACEDONIA_REPUBLIC_OF',
     'Poland',
     'Portugal',
     'Russia',
     'Scotland',
     'Slovakia',
     'Spain',
     'Sweden',
     'Switzerland',
     'Turkey',
     'Ukraine',
     'Wales'}

In [1]:
def cypherify(string, team = None):
    letters = list(string)

    if team:
        query = "MATCH p=(A:"+team+")"
        for i in range(len(string) - 1):
            query += "-[p" + str(i) + ":PASS]->(" + letters[i + 1] + ":"+team+")"
    else:
        query = "MATCH (A)"
        for i in range(len(string) - 1):
            query += "-[p" + str(i) + ":PASS]->(" + letters[i + 1] + ")"

    query += "\nWHERE "

    #correct order
    first = True
    for i in range(len(string) - 2):
        if first:
            query += "p" + str(i) + ".order + 1 = p" + str(i + 1) + ".order"
            first = False
        else:
            query += " and p" + str(i) + ".order + 1 = p" + str(i + 1) + ".order"

    #same possession
    first = True
    for i in range(len(string) - 2):
        if first:
            query += " and p" + str(i) + ".possession = p" + str(i + 1) + ".possession"
            first = False
        else:
            query += " and p" + str(i) + ".possession = p" + str(i + 1) + ".possession"

    #same match
    first = True
    for i in range(len(string) - 2):
        if first:
            query += " and p" + str(i) + ".match_id = p" + str(i + 1) + ".match_id"
            first = False
        else:
            query += " and p" + str(i) + ".match_id = p" + str(i + 1) + ".match_id"

    #different players
    unorderedPairGenerator = ((x, y) for x in set(letters) for y in set(letters) if y > x)
    if string != "AB":
        query += " and " + " and ".join([x + ".name <>" + " " +y + ".name" for x, y in list(unorderedPairGenerator)])
    else:
        query +=  " and ".join([x + ".name <>" + " " +y + ".name" for x, y in list(unorderedPairGenerator)])
    
    
    query += "\nRETURN COUNT(p)"
    return query

In [3]:
cypherify("AB", "ITALY")

'MATCH p=(A:ITALY)-[p0:PASS]->(B:ITALY)\nWHERE A.name <> B.name\nRETURN COUNT(p)'

In [191]:
result = {}
for key in all_teams():
    result[key.upper()] = {}

In [192]:
def get_value(pattern, team, app):
    query = cypherify(pattern, team)
    v = app.find_pattern(query)
    #print("Pattern: "+pattern+" \t Value: "+str(v))
    result[team][pattern] = v
    return v
    

In [193]:
team = "ITALY"
uri = "neo4j+s://00e145e7.databases.neo4j.io:7687"
user = "soccer_analytics"
password = "night-candle-miracle-nickel-declare-32"
app = App(uri, user, password)

def isNet(array):
    vect = [0]*len(array)
    return pass_net_check(1, len(array),vect, array)


def pass_net(i,n, vect, team, app):
    #print("------")
    
    if i==1:
        vect[i-1] = "A"
        pass_net(2,n,vect, team, app)
    elif i==2:
        vect[i-1] = "B"
        v = get_value("".join(vect[:i]), team, app)
        if v>0:
            pass_net(3,n,vect, team, app)
    elif i==n+1:
        #print("".join(vect))
        return
    else:
        for l in distinct_letters(vect, i-2):
            vect[i-1] = l
            v = get_value("".join(vect[:i]), team, app)
            if v > 0:
                pass_net(i+1,n,vect, team, app)
        vect[i-1] = first_new_letter(vect,i-1)
        v = get_value("".join(vect[:i]), team, app)
        if v > 0:
            pass_net(i+1,n,vect, team, app)

def first_new_letter(vect,end):
    letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "L", "M"]
    max_letter = max(vect[:end])
    return letters[letters.index(max_letter)+1]

def distinct_letters(vect, end):
    el = set(vect[:end])
    el.discard(vect[end])
    return list(el)

In [194]:

for team in all_teams():
    print("Team: "+team)
    team = team.upper()
    n = 6
    l = [0]*n
    pass_net(1, n, l, team, app)

Team: Portugal
Team: Scotland
Team: Italy
Team: England
Team: Spain
Team: Slovakia
Team: Wales
Team: Croatia
Team: MACEDONIA_REPUBLIC_OF
Team: Czech_Republic
Team: Turkey
Team: Poland
Team: Belgium
Team: Finland
Team: Germany
Team: Ukraine
Team: Austria
Team: Netherlands
Team: Hungary
Team: Sweden
Team: Switzerland
Team: Denmark
Team: France
Team: Russia


In [195]:
result

{'PORTUGAL': {'AB': 2117,
  'ABA': 312,
  'ABAB': 54,
  'ABABA': 10,
  'ABABAB': 2,
  'ABABAC': 6,
  'ABABC': 36,
  'ABABCA': 5,
  'ABABCB': 6,
  'ABABCD': 19,
  'ABAC': 216,
  'ABACA': 31,
  'ABACAC': 4,
  'ABACAB': 6,
  'ABACAD': 19,
  'ABACB': 22,
  'ABACBA': 2,
  'ABACBC': 3,
  'ABACBD': 15,
  'ABACD': 122,
  'ABACDA': 5,
  'ABACDC': 16,
  'ABACDB': 9,
  'ABACDE': 70,
  'ABC': 1470,
  'ABCA': 121,
  'ABCAC': 14,
  'ABCACA': 3,
  'ABCACB': 4,
  'ABCACD': 6,
  'ABCAB': 17,
  'ABCABA': 2,
  'ABCABC': 3,
  'ABCABD': 8,
  'ABCAD': 76,
  'ABCADA': 10,
  'ABCADC': 9,
  'ABCADB': 5,
  'ABCADE': 40,
  'ABCB': 198,
  'ABCBA': 31,
  'ABCBAC': 2,
  'ABCBAB': 5,
  'ABCBAD': 23,
  'ABCBC': 38,
  'ABCBCA': 5,
  'ABCBCB': 7,
  'ABCBCD': 21,
  'ABCBD': 108,
  'ABCBDA': 7,
  'ABCBDC': 13,
  'ABCBDB': 13,
  'ABCBDE': 51,
  'ABCD': 890,
  'ABCDA': 67,
  'ABCDAC': 9,
  'ABCDAD': 7,
  'ABCDAB': 7,
  'ABCDAE': 35,
  'ABCDC': 115,
  'ABCDCA': 17,
  'ABCDCD': 23,
  'ABCDCB': 19,
  'ABCDCE': 42,
  'ABCDB': 

In [185]:
result

{'PORTUGAL': {'AB': 254,
  'ABA': 49,
  'ABC': 197,
  'ABAB': 4,
  'ABABA': 1,
  'ABABAB': 0,
  'ABABAC': 0,
  'ABABC': 3,
  'ABABCA': 0,
  'ABABCB': 0,
  'ABABCD': 2,
  'ABABCDA': 0,
  'ABABCDC': 1,
  'ABABCDB': 0,
  'ABAC': 33,
  'ABACA': 6,
  'ABACAC': 0,
  'ABACAB': 1,
  'ABACAD': 4,
  'ABACB': 1,
  'ABACBA': 0,
  'ABACBC': 0,
  'ABACBD': 1,
  'ABACD': 20,
  'ABACDA': 1,
  'ABACDC': 1,
  'ABACDB': 1,
  'ABACDE': 14,
  'ABCA': 15,
  'ABCAC': 4,
  'ABCACA': 1,
  'ABCACB': 0,
  'ABCACD': 2,
  'ABCAB': 3,
  'ABCABA': 0,
  'ABCABC': 0,
  'ABCABD': 2,
  'ABCAD': 6,
  'ABCADA': 0,
  'ABCADC': 1,
  'ABCADB': 0,
  'ABCADE': 4,
  'ABCB': 31,
  'ABCBA': 2,
  'ABCBAC': 0,
  'ABCBAB': 1,
  'ABCBAD': 1,
  'ABCBC': 6,
  'ABCBCA': 0,
  'ABCBCB': 2,
  'ABCBCD': 4,
  'ABCBD': 21,
  'ABCBDA': 0,
  'ABCBDC': 1,
  'ABCBDB': 6,
  'ABCBDE': 8,
  'ABCD': 122,
  'ABCDA': 4,
  'ABCDAC': 1,
  'ABCDAD': 1,
  'ABCDAB': 0,
  'ABCDAE': 2,
  'ABCDC': 18,
  'ABCDCA': 2,
  'ABCDCD': 2,
  'ABCDCB': 3,
  'ABCDCE': 11

In [196]:
import json
json.dump(result, open("sunburst_unconstrained.json", "w"))

In [187]:

patterns = []
prefixes = []
values = []
colors = []
for res in result["SPAIN"]:
    
        patterns.append(res)
        prefixes.append(res[:-1])
        values.append(result["SPAIN"][res])
        colors.append(res[0:4] if len(res)>=4 else  12)
        
prefixes[0] = ""

data = dict(
patterns= patterns,
prefixes= prefixes,
values= values
)

In [188]:
colors

[12,
 12,
 12,
 'ABAB',
 'ABAB',
 'ABAB',
 'ABAB',
 'ABAB',
 'ABAB',
 'ABAB',
 'ABAB',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABAC',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCA',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCB',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'ABCD']

In [189]:
import plotly.express as px


fig = px.sunburst(data,
    names="patterns",
    parents="prefixes",
    values="values",
    color = prefixes
)
fig.show()